# 🌎 Welcome to the CSE151B Spring 2025 Climate Emulation Competition!

Thank you for participating in this exciting challenge focused on building machine learning models to emulate complex climate systems.  
This notebook is provided as a **starter template** to help you:

- Understand how to load and preprocess the dataset  
- Construct a baseline model  
- Train and evaluate predictions using a PyTorch Lightning pipeline  
- Format your predictions for submission to the leaderboard  

You're encouraged to:
- Build on this structure or replace it entirely
- Try more advanced models and training strategies
- Incorporate your own ideas to push the boundaries of what's possible

If you're interested in developing within a repository structure and/or use helpful tools like configuration management (based on Hydra) and logging (with Weights & Biases), we recommend checking out the following Github repo. Such a structure can be useful when running multiple experiments and trying various research ideas.

👉 [https://github.com/salvaRC/cse151b-spring2025-competition](https://github.com/salvaRC/cse151b-spring2025-competition)

Good luck, have fun, and we hope you learn a lot through this process!


In [ ]:
# run this cell if:
# (i) you are running this on colab, and
# (ii) have already saved the data on your gdrive.
# data is downloaded from kaggle

# from google.colab import drive
# drive.mount('/content/drive')

# !cp -r /content/drive/MyDrive/cse151b_data /content/

## Setting up WandB

In [ ]:
!pip install wandb

import wandb
wandb.login()
# the cell will give you a link (https://wandb.ai/authorize) and pause
# click on the link, sign in, copy your API key and paste it in here

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kac019 (cse151b) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(
    entity="cse151b",                 # our team name on WandB
    project="climate-project",        # let's stick to this name
    name="best_convlstm_optuna_scaled",                   # CHANGE THIS to whatever you want to identify the run with
    log_model="all"                   # logs everything (metrics, hyperparameters, weights)
)

### 📦 Install Required Libraries
We install the necessary Python packages for data loading, deep learning, and visualization.


In [ ]:
!pip install xarray zarr dask lightning matplotlib wandb cftime einops --quiet
!pip install optuna
import torch
import os
torch.use_deterministic_algorithms(False)

import os
from datetime import datetime
import numpy as np
import xarray as xr
import dask.array as da
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import lightning.pytorch as pl

### ⚙️ Configuration Setup  
Define all model, data, and training hyperparameters in one place for easy control and reproducibility.

### 📊 Data Configuration

We define the dataset settings used for training and evaluation. This includes:

- **`path`**: Path to the `.zarr` dataset containing monthly climate variables from CMIP6 simulations.
- **`input_vars`**: Climate forcing variables (e.g., CO₂, CH₄) used as model inputs.
- **`output_vars`**: Target variables to predict — surface air temperature (`tas`) and precipitation (`pr`).
- **`target_member_id`**: Ensemble member to use from the simulations (each SSP has 3) for target variables.
- **`train_ssps`**: SSP scenarios used for training (low to high emissions).
- **`test_ssp`**: Scenario held out for evaluation (Must be set to SSP245).
- **`test_months`**: Number of months to include in the test split (Must be set to 120).
- **`batch_size`** and **`num_workers`**: Data loading parameters for PyTorch training.

These settings reflect how the challenge is structured: models must learn from some emission scenarios and generalize to unseen ones.

> ⚠️ **Important:** Do **not modify** the following test settings:
>
> - `test_ssp` must remain **`ssp245`**, which is the held-out evaluation scenario.
> - `test_months` must be **`120`**, corresponding to the last 10 years (monthly resolution) of the scenario.



In [ ]:
config = {
    "data": {
        "path": "processed_data_cse151b_v2_corrupted_ssp245/processed_data_cse151b_v2_corrupted_ssp245.zarr",
        "input_vars": ["CO2", "SO2", "CH4", "BC", "rsdt"],
        "output_vars": ["tas", "pr"],
        "target_member_id": 0,
        "train_ssps": ["ssp126", "ssp370", "ssp585"],
        "test_ssp": "ssp245",
        "test_months": 360+12-1,
        "batch_size": 16,  # Slightly smaller batch due to time dimension
        "num_workers": 2,
    },
    "model": {
        "type": "convlstm",
        "hidden_dim": 64,
        "kernel_size": 3,
        "dropout_rate": 0.2,
        "seq_len": 24,  # Number of past months to stack
    },
    "training": {
        "lr": 1e-3,
    },
    "trainer": {
        "max_epochs": 80,
        "accelerator": "auto",
        "devices": "auto",
        "precision": 32,
        "num_sanity_val_steps": 0,
    },
    "seed": 42,
}
pl.seed_everything(config["seed"])

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

### 🔧 Spatial Weighting Utility Function

This cell sets up utility functions for reproducibility and spatial weighting:

- **`get_lat_weights(latitude_values)`**: Computes cosine-based area weights for each latitude, accounting for the Earth's curvature. This is critical for evaluating global climate metrics fairly — grid cells near the equator represent larger surface areas than those near the poles.


In [ ]:
def get_lat_weights(latitude_values):
    lat_rad = np.deg2rad(latitude_values)
    weights = np.cos(lat_rad)
    return weights / np.mean(weights)

### 🧠 SimpleCNN: A Residual Convolutional Baseline

This is a lightweight baseline model designed to capture spatial patterns in global climate data using convolutional layers.

- The architecture starts with a **convolution + batch norm + ReLU** block to process the input channels.
- It then applies a series of **residual blocks** to extract increasingly abstract spatial features. These help preserve gradient flow during training.
- Finally, a few convolutional layers reduce the feature maps down to the desired number of output channels (`tas` and `pr`).

This model only serves as a **simple baseline for climate emulation**.

We encourage you to build and experiment with your own models and ideas.


In [ ]:
def custom_pad(tensor):
    """
    Pads a tensor of shape [B, C, H, W] by:
    - Wrapping the last column to the left, and the first column to the right.
    - Averaging top two rows for top padding, and bottom two rows for bottom padding.
    """
    B, C, H, W = tensor.shape
    # Wrap columns
    left_pad = tensor[:, :, :, -1:].clone()  # last column
    right_pad = tensor[:, :, :, :1].clone()  # first column
    tensor = torch.cat([left_pad, tensor, right_pad], dim=3)  # pad W

    # Compute mean over width (dim=3), keeping B and C separate
    top_row_mean = tensor[:, :, 0, :].mean(dim=2, keepdim=True)  # shape [B, C, 1]
    bottom_row_mean = tensor[:, :, -1, :].mean(dim=2, keepdim=True)  # shape [B, C, 1]

    # Expand to shape [B, C, 1, W] to match row shape
    top_pad = top_row_mean.unsqueeze(2).expand(-1, -1, 1, tensor.shape[3])
    bottom_pad = bottom_row_mean.unsqueeze(2).expand(-1, -1, 1, tensor.shape[3])

    # Concatenate along height (H) dimension
    tensor = torch.cat([top_pad, tensor, bottom_pad], dim=2)
    return tensor

In [ ]:
class RowwiseConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, height, width, kernel_size=3):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.height = height

        self.scale_up_co2 = nn.Linear(1, height * width)
        self.scale_up_ch4 = nn.Linear(1, height * width)

        # One convolution per row
        self.convs = nn.ModuleList([
            nn.Conv2d(
                in_channels=input_dim + hidden_dim,
                out_channels=4 * hidden_dim,
                kernel_size=kernel_size,
                padding=0
            ) for _ in range(height)
        ])

    def forward(self, x, h, c):
        B, _, H, W = x.shape  # H = height

        CO2_scalar = x[:, 0:1, 0, 0]  # shape: [B, 1]
        CH4_scalar = x[:, 2:3, 0, 0]

        CO2 = self.scale_up_co2(CO2_scalar).view(B, 1, H, W)
        CH4 = self.scale_up_ch4(CH4_scalar).view(B, 1, H, W)

        SO2 = x[:, 1:2, :, :]
        BC = x[:, 3:4, :, :]
        rsdt = x[:, 4:, :, :]

        x = torch.cat((CO2, SO2, CH4, BC, rsdt), dim=1)
        combined = torch.cat([x, h], dim=1)  # shape: [B, C, H, W]

        combined = custom_pad(combined)

        # Apply row-wise convolutions
        outputs = []
        for row in range(H):
            row_input = combined[:, :, row:row+3, :]  # shape: [B, C, W]
            conv_out = self.convs[row](row_input)  # [B, 4*hidden_dim, W]
            outputs.append(conv_out)  # [B, 4*hidden_dim, 1, W]
        gates = torch.cat(outputs, dim=2)  # [B, 4*hidden_dim, H, W]
        i, f, o, g = torch.chunk(gates, 4, dim=1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

class ConvLSTMNet(nn.Module):
    def __init__(self, n_input_channels, n_output_channels, hidden_dim=64, kernel_size=3, dropout_rate=0.2, num_layers=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.conv_lstm = RowwiseConvLSTMCell(n_input_channels, hidden_dim, 48, 72, kernel_size)
        self.bn1 = nn.BatchNorm2d(hidden_dim+5)
        self.bn2 = nn.BatchNorm2d(hidden_dim)
        self.bn3 = nn.BatchNorm2d(hidden_dim)
        self.bn4 = nn.BatchNorm2d(hidden_dim)
        self.bn5 = nn.BatchNorm2d(hidden_dim)
        self.bn6 = nn.BatchNorm1d(hidden_dim)
        self.bn7 = nn.BatchNorm1d(hidden_dim)


        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout2d(dropout_rate)

        # self.conv_1 = nn.Conv2d(hidden_dim+5, hidden_dim*8, kernel_size=3, padding=0)
        # self.conv_2 = nn.Conv2d(hidden_dim*8, hidden_dim*8, kernel_size=3, padding=0)
        # self.conv_3 = nn.Conv2d(hidden_dim*8, hidden_dim*8, kernel_size=3, padding=0)
        # self.conv_4 = nn.Conv2d(hidden_dim*8, hidden_dim*8, kernel_size=3, padding=0)
        self.convs1 = nn.ModuleList([
            nn.Conv2d(
                in_channels=hidden_dim+5,
                out_channels=hidden_dim,
                kernel_size=3,
                padding=0
            ) for _ in range(48)
        ])
        self.convs2 = nn.ModuleList([
            nn.Conv2d(
                in_channels= hidden_dim,
                out_channels= hidden_dim,
                kernel_size=3,
                padding=0
            ) for _ in range(48)
        ])
        self.convs3 = nn.ModuleList([
            nn.Conv2d(
                in_channels=hidden_dim,
                out_channels=hidden_dim,
                kernel_size=3,
                padding=0
            ) for _ in range(48)
        ])

        self.scale_up_co2 = nn.Linear(hidden_dim, 48 * 72)
        self.scale_up_ch4 = nn.Linear(hidden_dim, 48 * 72)
        self.scale_up_co2_first = nn.Linear(1, hidden_dim)
        self.scale_up_ch4_first = nn.Linear(1, hidden_dim)

        self.conv_out = nn.Conv2d(hidden_dim, n_output_channels, kernel_size=1)


    def forward(self, x):
        # x: [B, T, C, H, W]
        B, T, C, H, W = x.shape
        h = torch.zeros(B, self.hidden_dim, H, W, device=x.device)
        c = torch.zeros(B, self.hidden_dim, H, W, device=x.device)
        for t in range(T):
            h, c = self.conv_lstm(x[:, t], h, c)

        CO2_scalar = x[:, T-1,0:1, 0, 0]  # shape: [B, 1]
        CH4_scalar = x[:, T-1,2:3, 0, 0]
        SO2 = x[:, T-1, 1:2, :, :]
        BC = x[:, T-1, 3:4, :, :]
        rsdt = x[:, T-1, 4:, :, :]

        CO2 = self.scale_up_co2_first(CO2_scalar)
        CH4 = self.scale_up_ch4_first(CH4_scalar)

        CO2 = self.relu(self.bn6(CO2))
        CH4 = self.relu(self.bn7(CH4))

        CO2 = self.scale_up_co2(CO2).view(B, 1, H, W)
        CH4 = self.scale_up_ch4(CH4).view(B, 1, H, W)

        x = torch.cat((CO2, SO2, CH4, BC, rsdt), dim=1)
        h = torch.cat([x, h], dim=1)  # shape: [B, C, H, W]


        h = self.relu(self.bn1(h))

        h = custom_pad(h)
        outputs = []
        for row in range(H):
            row_input = h[:, :, row:row+3, :]  # shape: [B, C, W]
            conv_out = self.convs1[row](row_input)  # [B, 4*hidden_dim, W]
            outputs.append(conv_out)
        h = torch.cat(outputs, dim=2)

        h = self.relu(self.bn2(h))
        h = custom_pad(h)
        outputs = []
        for row in range(H):
            row_input = h[:, :, row:row+3, :]  # shape: [B, C, W]
            conv_out = self.convs2[row](row_input)  # [B, 4*hidden_dim, W]
            outputs.append(conv_out)
        h = torch.cat(outputs, dim=2)  # [B, 4*hidden_dim, H, W]

        h = self.relu(self.bn4(h))
        h = custom_pad(h)
        outputs = []

        for row in range(H):
            row_input = h[:, :, row:row+3, :]  # shape: [B, C, W]
            conv_out = self.convs3[row](row_input)  # [B, 4*hidden_dim, W]
            outputs.append(conv_out)
        h = torch.cat(outputs, dim=2)

        # h = self.relu(self.bn5(h))
        # h = custom_pad(h)
        # h = self.conv_4(h)

        h = self.relu(self.bn3(h))
        h = self.conv_out(h)
        return h


### 📐 Normalizer: Z-Score Scaling for Climate Inputs & Outputs

This class handles **Z-score normalization**, a crucial preprocessing step for stable and efficient neural network training:

- **`set_input_statistics(mean, std)` / `set_output_statistics(...)`**: Store the mean and standard deviation computed from the training data for later use.
- **`normalize(data, data_type)`**: Standardizes the data using `(x - mean) / std`. This is applied separately to inputs and outputs.
- **`inverse_transform_output(data)`**: Converts model predictions back to the original physical units (e.g., Kelvin for temperature, mm/day for precipitation).

Normalizing the data ensures the model sees inputs with similar dynamic ranges and avoids biases caused by different variable scales.


In [ ]:
class Normalizer:
    def __init__(self):
        self.input_means = {}
        self.input_stds = {}
        self.output_means = {}
        self.output_stds = {}

    def normalize(self, data, data_type, var):
        if data_type == "input":
            return (data - self.input_means[var]) / self.input_stds[var]
        elif data_type == "output":
            return (data - self.output_means[var]) / self.output_stds[var]

    def inverse_transform_output(self, data):
        data[:,0,:,:] = data[:,0,:,:] * self.output_stds["tas"] + self.output_means["tas"]
        data[:,1,:,:] = data[:,1,:,:] * self.output_stds["pr"] + self.output_means["pr"]
        return data


### 🌍 Data Module: Loading, Normalization, and Splitting

This section handles the entire data pipeline, from loading the `.zarr` dataset to preparing PyTorch-ready DataLoaders.

#### `ClimateDataset`
- A simple PyTorch `Dataset` wrapper that preloads the entire (normalized) dataset into memory using Dask.
- Converts the data to PyTorch tensors and handles any `NaN` checks up front.

#### `ClimateDataModule`
A PyTorch Lightning `DataModule` that handles:
- ✅ **Loading data** from different SSP scenarios and ensemble members
- ✅ **Broadcasting non-spatial inputs** (like CO₂) to match spatial grid size
- ✅ **Normalization** using mean/std computed from training data only
- ✅ **Splitting** into training, validation, and test sets:
  - Training: All months from selected SSPs (except last 10 years of SSP370)
  - Validation: Last 10 years (120 months) of SSP370
  - Test: Last 10 years of SSP245 (unseen scenario)
- ✅ **Batching** and parallelized data loading via PyTorch `DataLoader`s
- ✅ **Latitude-based area weighting** for fair climate metric evaluation
- Shape of the inputs are Batch_Size X 5 (num_input_variables) X 48 X 72
- Shape of ouputputs are Batch_Size X 2 (num_output_variables) X 48 X 72

> ℹ️ **Note:** You likely won’t need to modify this class but feel free to make modifications if you want to inlcude different ensemble mebers to feed more data to your models


In [ ]:
class ClimateDataset(Dataset):
    def __init__(self, input_array, output_array=None, seq_len=48):
        self.seq_len = seq_len
        self.inputs = []
        self.outputs = [] if output_array is not None else None
        self.index_map = []  # maps global index to (chunk_id, local_index)

        self.size = 0
        for i in range(len(input_array)):
            input_tensor = torch.from_numpy(input_array[i]).float()

            if output_array is not None:
                output_tensor = torch.from_numpy(output_array[i]).float()
                if torch.isnan(input_tensor).any() or torch.isnan(output_tensor).any():
                    raise ValueError(f"NaNs found in dataset chunk {i}")
            else:
                if torch.isnan(input_tensor).any():
                    raise ValueError(f"NaNs found in input chunk {i}")
                output_tensor = None

            num_samples = input_tensor.shape[0] - seq_len + 1
            for j in range(num_samples):
                self.index_map.append((i, j))

            self.inputs.append(input_tensor)
            if self.outputs is not None:
                self.outputs.append(output_tensor)

            self.size += num_samples

        print(f"Creating dataset with {self.size} temporal samples...")

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        i, j = self.index_map[idx]
        x_seq = self.inputs[i][j:j + self.seq_len]  # shape: [T, C, H, W]
        if self.outputs is not None:
            y = self.outputs[i][j + self.seq_len - 1]  # shape: [C, H, W]
            return x_seq, y
        else:
            return x_seq


class ClimateDataModule(pl.LightningDataModule):
    def __init__(
        self,
        path,
        input_vars,
        output_vars,
        train_ssps,
        test_ssp,
        target_member_id,
        val_split=0.1,
        test_months=120,
        batch_size=32,
        num_workers=0,
        seed=42,
        seq_len=48,
    ):
        super().__init__()
        self.path = path
        self.input_vars = input_vars
        self.output_vars = output_vars
        self.train_ssps = train_ssps
        self.test_ssp = test_ssp
        self.target_member_id = target_member_id
        self.val_split = val_split
        self.test_months = test_months
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.seed = seed
        self.normalizer = Normalizer()
        self.seq_len = seq_len

    def prepare_data(self):
        assert os.path.exists(self.path), f"Data path not found: {self.path}"

    def setup(self, stage=None):
        ds = xr.open_zarr(self.path, consolidated=False, chunks={"time": 24})
        spatial_template = ds["rsdt"].isel(time=0, ssp=0, drop=True)

        def load_ssp(ssp):
            input_dask, output_dask = [], []
            for var in self.input_vars:
                da_var = ds[var].sel(ssp=ssp)
                if "latitude" in da_var.dims:
                    da_var = da_var.rename({"latitude": "y", "longitude": "x"})
                if "member_id" in da_var.dims:
                    da_var = da_var.sel(member_id=self.target_member_id)
                if set(da_var.dims) == {"time"}:
                    da_var = da_var.broadcast_like(spatial_template).transpose("time", "y", "x")
                input_dask.append(da_var.data)

            for var in self.output_vars:
                da_out = ds[var].sel(ssp=ssp, member_id=self.target_member_id)
                if "latitude" in da_out.dims:
                    da_out = da_out.rename({"latitude": "y", "longitude": "x"})
                output_dask.append(da_out.data)

            return da.stack(input_dask, axis=1), da.stack(output_dask, axis=1)

        def to_numpy(da_array):
            return da_array.compute().astype(np.float32)

        def make_numpy(normalized_data):
            for i in range(len(normalized_data)):
                normalized_data[i] = to_numpy(normalized_data[i])
            return normalized_data

        def pad_sequence_start(x, pad_len):
            pad = np.repeat(x[[0]], pad_len, axis=0)  # Repeat first frame
            return np.concatenate([pad, x], axis=0)

        train_input, train_output, val_input, val_output = [], [], [], []

        for ssp in self.train_ssps:
            x, y = load_ssp(ssp)
            if ssp == "ssp370":
                val_input.append(x[-self.test_months:])
                val_output.append(y[-self.test_months:])
                train_input.append(x[:-self.test_months])
                train_output.append(y[:-self.test_months])
            else:
                train_input.append(x)
                train_output.append(y)
        train_input_list = train_input
        train_output_list = train_output
        val_input_list = val_input
        val_output_list = val_output

        input_mean = 0.0
        input_std = 0.0

        eps = 1e-6  # avoid log(0)
        for i in range(len(train_input_list)):
            data = train_input_list[i]

            def transform_channels(x):
                # x is a NumPy array with shape [samples, channels, height, width]
                x1 = x[:, 0:1]  # keep dims
                x2 = np.log(x[:, 1:2] + eps)
                x3 = x[:, 2:3]
                x4 = np.log(x[:, 3:4] + eps)
                x5 = x[:, 4:5]
                return np.concatenate([x1, x2, x3, x4, x5], axis=1)

            data = data.map_blocks(transform_channels, dtype=data.dtype)
            train_input_list[i] = data

            input_mean += da.nanmean(data, axis=(0, 2, 3), keepdims=True).compute()
            input_std  += da.nanstd(data, axis=(0, 2, 3), keepdims=True).compute()

        input_mean/=3
        input_std /=3

        input_means = {}
        input_stds = {}
        for v, var in enumerate(self.input_vars):
            input_means[var] = input_mean[0,v,0,0]
            input_stds[var] = input_std[0,v,0,0]

        self.normalizer.input_means = input_means
        self.normalizer.input_stds = input_stds

        output_mean = 0.0
        output_std = 0.0

        for i in range(len(train_output_list)):
            output_mean += da.nanmean(train_output_list[i], axis=(0, 2, 3), keepdims=True).compute()
            output_std += da.nanstd(train_output_list[i], axis=(0, 2, 3), keepdims=True).compute()

        output_mean /= 3
        output_std /= 3

        output_means = {}
        output_stds = {}
        for v, var in enumerate(self.output_vars):
            output_means[var] = output_mean[0, v, 0, 0]
            output_stds[var] = output_std[0, v, 0, 0]

        self.normalizer.output_means = output_means
        self.normalizer.output_stds = output_stds


        normalized_train_inputs = []

        for i in range(len(train_input_list)):
            data = train_input_list[i]  # shape: [samples, channels, H, W]
            var_list = []

            for v, var in enumerate(self.input_vars):
                channel_data = data[:, v:v+1, :, :]  # keep shape [samples, 1, H, W]
                normed = self.normalizer.normalize(channel_data, "input", var)
                var_list.append(normed)

            # Concatenate normalized channels back along the channel axis
            normalized_data = da.concatenate(var_list, axis=1)  # shape: [samples, channels, H, W]
            normalized_train_inputs.append(normalized_data)

        normalized_train_outputs = []

        for i in range(len(train_output_list)):
            data = train_output_list[i]
            var_list = []

            for v, var in enumerate(self.output_vars):
                channel_data = data[:, v:v+1, :, :]
                normed = self.normalizer.normalize(channel_data, "output", var)
                var_list.append(normed)

            normalized_data = da.concatenate(var_list, axis=1)
            normalized_train_outputs.append(normalized_data)

        normalized_val_inputs = []

        for i in range(len(val_input_list)):
            data = val_input_list[i]
            var_list = []

            for v, var in enumerate(self.input_vars):
                channel_data = data[:, v:v+1, :, :]
                normed = self.normalizer.normalize(channel_data, "input", var)
                var_list.append(normed)

            normalized_data = da.concatenate(var_list, axis=1)
            normalized_val_inputs.append(normalized_data)

        normalized_val_outputs = []

        for i in range(len(val_output_list)):
            data = val_output_list[i]
            var_list = []

            for v, var in enumerate(self.output_vars):
                channel_data = data[:, v:v+1, :, :]
                normed = self.normalizer.normalize(channel_data, "output", var)
                var_list.append(normed)

            normalized_data = da.concatenate(var_list, axis=1)
            normalized_val_outputs.append(normalized_data)


        test_input, test_output = load_ssp(self.test_ssp)
        test_input = test_input[-self.test_months:]
        test_output = test_output[-self.test_months:]
        test_input_list = [test_input]
        test_output_list = [test_output]

        normalized_test_inputs=[]

        for i in range(len(test_input_list)):
                    data = test_input_list[i]
                    var_list = []

                    for v, var in enumerate(self.input_vars):
                        channel_data = data[:, v:v+1, :, :]
                        normed = self.normalizer.normalize(channel_data, "input", var)
                        var_list.append(normed)

                    normalized_data = da.concatenate(var_list, axis=1)
                    normalized_test_inputs.append(normalized_data)
        normalized_test_outputs = []

        for i in range(len(test_output_list)):
            data = test_output_list[i]
            var_list = []

            for v, var in enumerate(self.output_vars):
                channel_data = data[:, v:v+1, :, :]
                normed = self.normalizer.normalize(channel_data, "output", var)
                var_list.append(normed)

            normalized_data = da.concatenate(var_list, axis=1)
            normalized_test_outputs.append(normalized_data)


        train_input_list = make_numpy(normalized_train_inputs)
        train_output_list = make_numpy(normalized_train_outputs)
        val_input_list = make_numpy(normalized_val_inputs)
        val_output_list = make_numpy(normalized_val_outputs)
        test_input_list = make_numpy(normalized_test_inputs)
        test_output_list = make_numpy(normalized_test_outputs)

        self.train_dataset = ClimateDataset(train_input_list, train_output_list, seq_len=self.seq_len)
        self.val_dataset = ClimateDataset(val_input_list, val_output_list, seq_len=self.seq_len)
        self.test_dataset = ClimateDataset(test_input_list, test_output_list, seq_len=self.seq_len)

        self.lat = spatial_template.y.values
        self.lon = spatial_template.x.values
        self.area_weights = xr.DataArray(get_lat_weights(self.lat), dims=["y"], coords={"y": self.lat})

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True,
                          num_workers=self.num_workers, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def get_lat_weights(self):
        return self.area_weights

    def get_coords(self):
        return self.lat, self.lon

In [ ]:
print(datamodule.test_dataset.outputs[0].shape)
print(datamodule.test_dataset.__getitem__(0)[0].shape)

torch.Size([383, 2, 48, 72])
torch.Size([48, 5, 48, 72])


### ⚡ ClimateEmulationModule: Lightning Wrapper for Climate Model Emulation

This is the core model wrapper built with **PyTorch Lightning**, which organizes the training, validation, and testing logic for the climate emulation task. Lightning abstracts away much of the boilerplate code in PyTorch-based deep learning workflows, making it easier to scale models.

#### ✅ Key Features

- **`training_step` / `validation_step` / `test_step`**: Standard Lightning hooks for computing loss and predictions at each stage. The loss used is **Mean Squared Error (MSE)**.

- **Normalization-aware outputs**:
  - During validation and testing, predictions and targets are denormalized before evaluation using stored mean/std statistics.
  - This ensures evaluation is done in real-world units (Kelvin and mm/day).

- **Metric Evaluation** via `_evaluate()`:
  For each variable (`tas`, `pr`), it calculates:
  - **Monthly Area-Weighted RMSE**
  - **Time-Mean RMSE** (RMSE on 10-year average's)
  - **Time-Stddev MAE** (MAE on 10-year standard deviation; a measure of temporal variability)
    
  These metrics reflect the competition's evaluation criteria and are logged and printed.

- **Kaggle Submission Writer**:
  After testing, predictions are saved to a `.csv` file in the required Kaggle format via `_save_submission()`.

- **Saving Predictions for Visualization**:
  - Validation predictions are saved tao `val_preds.npy` and `val_trues.npy`
  - These can be loaded later for visual inspection of the model's performance.

 🔧 **Feel free to modify any part of this module** (loss functions, evaluation, training logic) to better suit your model or training pipeline / Use pure PyTorch etc.

⚠️ The **final submission `.csv` file must strictly follow the format and naming convention used in `_save_submission()`**, as these `ID`s are used to match predictions to the hidden test set during evaluation.



In [ ]:
import pandas as pd

class ClimateEmulationModule(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-4):
        super().__init__()
        self.model = model
        self.save_hyperparameters(ignore=['model']) # Save all hyperparameters except the model to self.hparams.<param_name>
        self.criterion = nn.MSELoss()
        self.normalizer = None
        self.val_preds, self.val_targets = [], []
        self.test_preds, self.test_targets = [], []

    def forward(self, x):
        return self.model(x)

    def on_fit_start(self):
        self.normalizer = self.trainer.datamodule.normalizer  # Get the normalizer from the datamodule (see above)

    def training_step(self, batch, batch_idx):
        x, y = batch # Unpack inputs and targets (this is the output of the _getitem_ method in the Dataset above)
        y_hat = self(x)   # Forward pass
        loss = self.criterion(y_hat, y)  # Calculate loss
        self.log("train/loss", loss)  # Log loss for tracking
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val/loss", loss)

        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = self.normalizer.inverse_transform_output(y.detach().cpu().numpy())

        self.val_preds.append(y_hat_np)
        self.val_targets.append(y_np)

        return loss

    def on_validation_epoch_end(self):
        # Concatenate all predictions and ground truths from each val step/batch into one array
        preds = np.concatenate(self.val_preds, axis=0)
        trues = np.concatenate(self.val_targets, axis=0)
        self._evaluate(preds, trues, phase="val")
        np.save("val_preds.npy", preds)
        np.save("val_trues.npy", trues)
        self.val_preds.clear()
        self.val_targets.clear()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = y.detach().cpu().numpy()

        self.test_preds.append(y_hat_np)
        self.test_targets.append(y_np)

    def on_test_epoch_end(self):
        # Concatenate all predictions and ground truths from each test step/batch into one array
        preds = np.concatenate(self.test_preds, axis=0)
        trues = np.concatenate(self.test_targets, axis=0)
        self._evaluate(preds, trues, phase="test")
        self._save_submission(preds)
        self.test_preds.clear()
        self.test_targets.clear()

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

        scheduler = optim.lr_scheduler.StepLR(
        optimizer, step_size=10, gamma=0.5
    )

    def _evaluate(self, preds, trues, phase="val"):
        datamodule = self.trainer.datamodule
        area_weights = datamodule.get_lat_weights()
        lat, lon = datamodule.get_coords()
        time = np.arange(preds.shape[0])
        output_vars = datamodule.output_vars

        for i, var in enumerate(output_vars):
            p = preds[:, i]
            t = trues[:, i]
            p_xr = xr.DataArray(p, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})
            t_xr = xr.DataArray(t, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})

            # RMSE
            rmse = np.sqrt(((p_xr - t_xr) ** 2).weighted(area_weights).mean(("time", "y", "x")).item())
            # RMSE of time-mean
            mean_rmse = np.sqrt(((p_xr.mean("time") - t_xr.mean("time")) ** 2).weighted(area_weights).mean(("y", "x")).item())
            # MAE of time-stddev
            std_mae = np.abs(p_xr.std("time") - t_xr.std("time")).weighted(area_weights).mean(("y", "x")).item()

            print(f"[{phase.upper()}] {var}: RMSE={rmse:.4f}, Time-Mean RMSE={mean_rmse:.4f}, Time-Stddev MAE={std_mae:.4f}")
            self.log_dict({
                f"{phase}/{var}/rmse": rmse,
                f"{phase}/{var}/time_mean_rmse": mean_rmse,
                f"{phase}/{var}/time_std_mae": std_mae,
            })

    def _save_submission(self, predictions):
        datamodule = self.trainer.datamodule
        lat, lon = datamodule.get_coords()
        output_vars = datamodule.output_vars
        time = np.arange(predictions.shape[0])

        rows = []
        for t_idx, t in enumerate(time):
            for var_idx, var in enumerate(output_vars):
                for y_idx, y in enumerate(lat):
                    for x_idx, x in enumerate(lon):
                        row_id = f"t{t_idx:03d}_{var}_{y:.2f}_{x:.2f}"
                        pred = predictions[t_idx, var_idx, y_idx, x_idx]
                        rows.append({"ID": row_id, "Prediction": pred})

        df = pd.DataFrame(rows)
        os.makedirs("submissions", exist_ok=True)
        filepath = f"submissions/kaggle_submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filepath, index=False)
        print(f"✅ Submission saved to: {filepath}")

In [ ]:
#Integrating optuna for hyperparameter training
import optuna

def objective(trial):


    hidden_dim = trial.suggest_categorical("hidden_dim", [32, 64, 128])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    seq_len = trial.suggest_categorical("seq_len", [24, 36, 48, 72])
    num_layers = trial.suggest_int("num_layers", 1, 2)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])

    config["model"]["hidden_dim"] = hidden_dim
    config["model"]["dropout_rate"] = dropout_rate
    config["model"]["seq_len"] = seq_len
    config["model"]["num_layers"] = num_layers
    config["training"]["lr"] = lr
    config["data"]["batch_size"] = batch_size

    datamodule = ClimateDataModule(**config["data"], seq_len=seq_len)
    model = ConvLSTMNet(
        n_input_channels=len(config["data"]["input_vars"]),
        n_output_channels=len(config["data"]["output_vars"]),
        hidden_dim=hidden_dim,
        kernel_size=3,
        dropout_rate=dropout_rate,
        num_layers=num_layers
    )
    lightning_module = ClimateEmulationModule(model, learning_rate=lr)

    trainer = pl.Trainer(
        logger=wandb_logger,
        max_epochs=5,
        enable_progress_bar=False,
        enable_model_summary=False,
    )
    trainer.fit(lightning_module, datamodule=datamodule)

    # Get validation RMSE (tas)
    val_rmse = trainer.callback_metrics.get("val/tas/rmse")
    return val_rmse if val_rmse is not None else float("inf")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best hyperparameters:", study.best_params)
best_params = study.best_params
config["model"]["hidden_dim"] = best_params["hidden_dim"]
config["model"]["kernel_size"] = best_params["kernel_size"]
config["model"]["dropout_rate"] = best_params["dropout_rate"]
config["model"]["seq_len"] = best_params["seq_len"]
config["training"]["lr"] = best_params["lr"]
config["model"]["num_layers"] = best_params["num_layers"]

[I 2025-06-02 05:48:33,774] A new study created in memory with name: no-name-c7f8c7fe-35d1-4ea7-9652-53191c083bd8
INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 2025-06-02 05:48:37,374] Trial 0 failed with parameters: {'hidd

NameError: name 'exit' is not defined

### ⚡ Training & Evaluation with PyTorch Lightning

This block sets up and runs the training and testing pipeline using **PyTorch Lightning’s `Trainer`**, which abstracts away much of the boilerplate in deep learning workflows.

- **Modular Setup**:
  - `datamodule`: Handles loading, normalization, and batching of climate data.
  - `model`: A convolutional neural network that maps climate forcings to predicted outputs.
  - `lightning_module`: Wraps the model with training/validation/test logic and metric evaluation.

- **Trainer Flexibility**:
  The `Trainer` accepts a wide range of configuration options from `config["trainer"]`, including:
  - Number of epochs
  - Precision (e.g., 16-bit or 32-bit)
  - Device configuration (CPU, GPU, or TPU)
  - Determinism, logging, callbacks, and more

In [ ]:
from lightning.pytorch.callbacks import EarlyStopping

datamodule = ClimateDataModule(**config["data"])
config["model"]["hidden_dim"] = 128
config["model"]["dropout_rate"] = 0.2980183070100913
config["model"]["seq_len"] = 12
config["model"]["num_layers"] = 2
config["training"]["lr"] = 0.0005830015980382919
config["data"]["batch_size"] = 8

model_type = config["model"]["type"]

if model_type == "convlstm":
    model = ConvLSTMNet(
        n_input_channels=len(config["data"]["input_vars"]),
        n_output_channels=len(config["data"]["output_vars"]),
        hidden_dim=config["model"]["hidden_dim"],
        kernel_size=config["model"]["kernel_size"],
        dropout_rate=config["model"]["dropout_rate"],
        num_layers=config["model"]["num_layers"]
    )

lightning_module = ClimateEmulationModule(model, learning_rate=config["training"]["lr"])

early_stop_callback = EarlyStopping(
    monitor="val/loss",
    patience=10,
    mode="min",
    verbose=True
)

trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[early_stop_callback],
    **config["trainer"]
)

trainer.fit(lightning_module, datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory ./climate-project/o4v0wuqu/checkpoints exists and is not empty.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CU

Creating dataset with 2551 temporal samples...
Creating dataset with 324 temporal samples...
Creating dataset with 324 temporal samples...


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Metric val/loss improved. New best score: 0.199
INFO:lightning.pytorch.callbacks.early_stopping:Metric val/loss improved. New best score: 0.199


...

INFO: Metric val/loss improved by 0.009 >= min_delta = 0.0. New best score: 0.177
INFO:lightning.pytorch.callbacks.early_stopping:Metric val/loss improved by 0.009 >= min_delta = 0.0. New best score: 0.177


[VAL] tas: RMSE=1.9714, Time-Mean RMSE=1.3979, Time-Stddev MAE=0.4135
[VAL] pr: RMSE=2.0573, Time-Mean RMSE=0.4332, Time-Stddev MAE=0.8704


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Metric val/loss improved by 0.001 >= min_delta = 0.0. New best score: 0.175
INFO:lightning.pytorch.callbacks.early_stopping:Metric val/loss improved by 0.001 >= min_delta = 0.0. New best score: 0.175


[VAL] tas: RMSE=1.8074, Time-Mean RMSE=1.1973, Time-Stddev MAE=0.3967
[VAL] pr: RMSE=2.0649, Time-Mean RMSE=0.4738, Time-Stddev MAE=0.8448


In [ ]:
import wandb

# Finish any old run first (optional but recommended)
if wandb.run is not None:
    wandb.finish()

# Create logger
wandb_logger = WandbLogger(project="climate-project", name="best_convlstm_optuna")

# # Log parameter count
# wandb.init(project="climate-project", resume="allow", reinit=True)
# wandb.log({"parameter_count": parameter_count})


# Test model

**IMPORTANT:** Please note that the test metrics will be bad because the test targets have been corrupted on the public Kaggle dataset.
The purpose of testing below is to generate the Kaggle submission file based on your model's predictions, which you can submit to the competition.

In [ ]:
trainer.test(lightning_module, datamodule=datamodule)

### Plotting Utils


In [ ]:
def plot_comparison(true_xr, pred_xr, title, cmap='viridis', diff_cmap='RdBu_r', metric=None):
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))

    vmin = min(true_xr.min().item(), pred_xr.min().item())
    vmax = max(true_xr.max().item(), pred_xr.max().item())

    # Ground truth
    true_xr.plot(ax=axs[0], cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=True)
    axs[0].set_title(f"{title} (Ground Truth)")

    # Prediction
    pred_xr.plot(ax=axs[1], cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=True)
    axs[1].set_title(f"{title} (Prediction)")

    # Difference
    diff = pred_xr - true_xr
    abs_max = np.max(np.abs(diff))
    diff.plot(ax=axs[2], cmap=diff_cmap, vmin=-abs_max, vmax=abs_max, add_colorbar=True)
    axs[2].set_title(f"{title} (Difference) {f'- {metric:.4f}' if metric else ''}")

    plt.tight_layout()
    plt.show()


### 🖼️ Visualizing Validation Predictions

This cell loads saved validation predictions and compares them to the ground truth using spatial plots. These visualizations help you qualitatively assess your model's performance.

For each output variable (`tas`, `pr`), we visualize:

- **📈 Time-Mean Map**: The 10-year average spatial pattern for both prediction and ground truth. Helps identify long-term biases or spatial shifts.
- **📊 Time-Stddev Map**: Shows the standard deviation across time for each grid cell — useful for assessing how well the model captures **temporal variability** at each location.
- **🕓 Random Timestep Sample**: Visual comparison of prediction vs ground truth for a single month. Useful for spotting fine-grained anomalies or errors in specific months.

> These plots provide intuition beyond metrics and are useful for debugging spatial or temporal model failures.


In [ ]:
# Load validation predictions
# make sure to have run the validation loop at least once
val_preds = np.load("val_preds.npy")
val_trues = np.load("val_trues.npy")

lat, lon = datamodule.get_coords()
output_vars = datamodule.output_vars
time = np.arange(val_preds.shape[0])

for i, var in enumerate(output_vars):
    pred_xr = xr.DataArray(val_preds[:, i], dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})
    true_xr = xr.DataArray(val_trues[:, i], dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})

    # --- Time Mean ---
    plot_comparison(true_xr.mean("time"), pred_xr.mean("time"), f"{var} Val Time-Mean")

    # --- Time Stddev ---
    plot_comparison(true_xr.std("time"), pred_xr.std("time"), f"{var} Val Time-Stddev", cmap="plasma")

    # --- Random timestep ---
    t_idx = np.random.randint(0, len(time))
    plot_comparison(true_xr.isel(time=t_idx), pred_xr.isel(time=t_idx), f"{var} Val Sample Timestep {t_idx}")


In [ ]:
metrics = pd.read_csv("logs/climate/version_13/metrics.csv")

# Drop rows where val/loss is NaN (e.g., training-only steps)
train_loss = metrics[metrics["train/loss"].notna()]
val_loss = metrics[metrics["val/loss"].notna()]

plt.figure(figsize=(10, 5))
plt.plot(train_loss["step"], train_loss["train/loss"], label="Train Loss")
plt.plot(val_loss["step"], val_loss["val/loss"], label="Val Loss")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Training and Validation Loss over Steps")
plt.legend()
plt.grid(True)
plt.show()

## 🧪 Final Notes

This notebook is meant to serve as a **baseline template** — a starting point to help you get up and running quickly with the climate emulation challenge.

You are **not** required to stick to this exact setup. In fact, we **encourage** you to:

- 🔁 Build on top of the provided `DataModule`.
- 🧠 Use your own model architectures or training pipelines that you’re more comfortable with
- ⚗️ Experiment with ideas  
- 🥇 Compete creatively to climb the Kaggle leaderboard  
- 🙌 Most importantly: **have fun** and **learn as much as you can** along the way

This challenge simulates a real-world scientific problem, and there’s no single "correct" approach — so be curious, experiment boldly, and make it your own!
